# Multi-layer percepton in Tensorflow
In this notebook, we are going to implement a Multi-layer percepton (MLP) in tensorflow. First, let us import the MNIST dataset from the tensorflow example dataset, and define few parameters for the system.

In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("F:/Deep Learning/MNIST_data/", one_hot=True)


batchSize=100
maxIteration=int(mnist.train.images.shape[0]/batchSize)+1
total_epoch=20

Extracting F:/Deep Learning/MNIST_data/train-images-idx3-ubyte.gz
Extracting F:/Deep Learning/MNIST_data/train-labels-idx1-ubyte.gz
Extracting F:/Deep Learning/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting F:/Deep Learning/MNIST_data/t10k-labels-idx1-ubyte.gz


## Define MLP structure
The structure of the MLP is defined as follows,

In [16]:
imgSize=28
outputs=10
# Three hidden layers, input and output layers
inputs=imgSize*imgSize
layers=[inputs,400,200,100,outputs]

Define input and output placeholders of the MLP

In [17]:
xnode=tf.placeholder(tf.float32,[None,inputs])
ynode=tf.placeholder(tf.float32,[None,outputs])

Now, we define a function to set a single layers of percepton. We'll call this function to define each hidden layers and the output layer. Each layer contains wights (w) and bias (b). Hidden layers provide output after a ReLU unit, and output layer left it for the softmax function.

In [18]:
def single_percepton(x,n_inputs,n_nurons,hidden='Yes'):
    w=tf.Variable(tf.random_normal([n_inputs,n_nurons]))
    b=tf.Variable(tf.random_normal([n_nurons]))
    activation=tf.add(tf.matmul(x,w),b)
    if hidden=='Yes': return tf.nn.relu(activation)
    else: return activation

We call the single layer function multiple time to defien all layers as follows,

In [19]:
def multiple_percepton(x,layers):
    l=[x]      
    for i in range(len(layers)-1):
        if(i<len(layers)-2): l.append(single_percepton(l[i],layers[i],layers[i+1]))
        else:l.append(single_percepton(l[i],layers[i],layers[i+1],hidden='No'))      
    return l

We just receive output from all layers. Output from the last layer has been defined as the outputnode.

In [20]:
l=multiple_percepton(xnode,layers)
outnode=l[-1:]

## Define the cost function and performance measure
We are going to minimize average cross entropy between the estimation and the ground true. We also define the accuracy as the performance measure of our estiamtion.

In [21]:
#Cost function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=outnode, labels=ynode))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

#Performance measure
yestimate=tf.nn.softmax(outnode)
yestimate=tf.argmax(yestimate,dimension=2)
ytrue=tf.argmax(ynode,dimension=1)
correct_prediction=tf.equal(ytrue,yestimate)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

## Start Tensor Session
Now, we initialize our tensor session. Let us see the cost and accuracy on the test dataset. We'll examine it again after training.

In [22]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

x_test=mnist.test.images
y_test=mnist.test.labels

c,acc=sess.run([cost,accuracy],feed_dict={xnode:x_test,ynode:y_test})
print("\n(Before training) Test accuracy:{0:1.2%} Test Cost: {1:.5}".format(acc,c))


(Before training) Test accuracy:9.63% Test Cost: 1.8596e+04


Before training, the accuracy is very low. Let us train the MLP for multiple epochs.

In [23]:
costAll=[]
for epoch in range(total_epoch):
    for i in range(maxIteration):
        x_batch,y_batch=mnist.train.next_batch(batchSize)
        _,c=sess.run([optimizer,cost],feed_dict={xnode:x_batch,ynode:y_batch})
        costAll.append(c)
    print("Epoch: {0:2} Training Cost: {1:.5}".format(epoch+1,c))

Epoch:  1 Training Cost: 201.76


KeyboardInterrupt: 

Accuracy on the test dataset should significantly imporive after training.

In [ ]:
c,acc=sess.run([cost,accuracy],feed_dict={xnode:x_test,ynode:y_test})
print("\n(After training) Test accuracy:{0:1.2%} Test Cost: {1:.5}".format(acc,c))

The training cost is also minimized over iteration.

In [ ]:
plt.plot(costAll)
plt.xlabel('Iteration (all epoch)')
plt.ylabel('Cost')
plt.title('Training cost over iteration')
plt.show()

We can examine the structure of the MLP system from the outputs for all layers.

In [ ]:
l